## 领域驱动设计中的战术设计

DDD 中战术模式的作用是管理复杂性并确保领域模型中行为的清晰性。

每种战术模式（`building block pattern`）都是为了解决特定的一类问题。

像 `value object` 还有 `entity` 这种并不是 DDD 的专有名词，只是在 DDD 这个语境 (类比于 `Bounded Context`) 下又有了自己的含义。

### Value Object

我们一般用 `Value Object` 来表示没有唯一标识符（`identity`）的概念，比如钱、价格这种。

钱这种概念大家应该再熟悉不过了，假设我有一张一百块的人民币，你也有一百块的人民币，在这个情境下，人民币的新旧、编号都不重要，那么它们在 DDD 的概念里是相等的。

比较值对象是一个关键操作，尽管它们没有唯一标识符，我们使用基于属性/值（`attribute-based equality`/`value-based equality`）来做对比。

还有个比较重要的特征就是值对象是不变的（`Immutable`）。

如果用 python 的话，可以用 `dataclass` 这个模块来实现 Value Object。

In [1]:
from dataclasses import dataclass
from decimal import Decimal

@dataclass(frozen=True)
class Money:
    amount: Decimal


m1 = Money(Decimal(100))
m2 = Money(Decimal(100))

assert m1 == m2

值对象需要体现丰富的领域知识，DDD 社区里一般喜欢用值对象来取代原始类型，然后要体现内聚性。最直观的可能就是为这个值对象添加方法或者属性。

例如钱和钱可以相加，相减等操作，但是因为值对象是不可变的，所以生成的都是新的值对象。

In [2]:
from typing import Union
from dataclasses import dataclass
from decimal import Decimal


@dataclass(frozen=True)
class Money:
    amount: Decimal

    @classmethod
    def of(cls, amount: Union[int, str, float]) -> 'Money':
        return Money(Decimal(amount))

    def __add__(self, other: 'Money') -> 'Money':
        return Money(self.amount + other.amount)

    def __sub__(self, other: 'Money') -> 'Money':
        return Money(self.amount - other.amount)

    def __repr__(self) -> str:
        return f'{self.amount:.2f}'

m1 = Money.of(100)
m2 = Money.of(99.5)

print(m1 + m2)
print(m1 - m2)

199.50
0.50


### Entity

> Entities are fundamentally about identity—focusing on the “who” rather than the “what.”

与 `value object` 不同，`entity`（实体）对象会有唯一标识符，但是定义某个概念是值对象还是实体，还是需要从上下文出发（`context-dependent`）。一个概念在某个上下文中是个实体在另一个上下文中可能是个值对象。

通常来说实体的很多属性类型都是值对象，值对象的属性类型也可能是值对象。正如我们上文所说，值对象需要体现丰富的领域知识，结合 `SRP` 原则，我们一般选择将实体的大量业务 `invariants` 封装到值对象中，给一个验证的例子会更加直观，后续会有专门的文章谈一下我对验证的理解和最佳实践。

#### 实体的唯一标识符

实体的唯一标识符，可以是自然键(`Natural key`) 也可以是代理键(`Surrogate key`)，甚至可能是复合键(`Composite key`)。

其实这几种键用过关系型数据库的大家都很容易理解。

自然键典型的例子有身份证号、ISBN 等，这种键跟业务的关系很紧密，也十分直观。

In [3]:
from dataclasses import dataclass, field

@dataclass(frozen=True)
class ISBN:
    number: str

@dataclass
class Book:
    id: str = field(init=False)
    isbn: ISBN

    def __post_init__(self) -> None:
        self.id = self.isbn.number

    def __repr__(self) -> str:
        return f'Book({self.id})'
    
    def __eq__(self, other: 'Book') -> bool:
        return self.id == other.id

book1 = Book(ISBN('903121212121'))
book2 = Book(ISBN('903121212123'))
book3 = Book(ISBN('903121212121'))
print(book1, book2, book3)
assert book1 != book2
assert book1 == book3

Book(903121212121) Book(903121212123) Book(903121212121)


代理键大家用的也比较多了，主要特征就是没有业务属性，比如 MySQL 的自增主键(auto_increment) 或者 UUID / GUID 。

In [4]:
import uuid
from dataclasses import dataclass

@dataclass
class Hotel:
    id: str = field(init=False)

    def __post_init__(self) -> None:
        self.id: str = str(uuid.uuid4())

还有一种是组合键，这种键可能就是用实体的几个字段组合成一个字符串来表示唯一标识符，也体现业务特征，但是往往生成比较麻烦，而且只要业务属性稍有调整就有大麻烦，所以我一般不推荐这种方式。

关于这些键的区别和联系还有生产中性能/用户体验各方面的问题，[Choosing A Primary Key: Natural Or Surrogate?](https://agiledata.org/essays/keys.html) 可能是个不错的入门指南。

## 参考资料
* [collections-primitive-obsession](https://enterprisecraftsmanship.com/posts/collections-primitive-obsession/)

* [value-objects-with-python](https://blog.szymonmiks.pl/p/value-objects-with-python/)